**Nota**: Instalar las siguientes bibliotecas antes de importarlas por primera vez

In [ ]:
%%capture
# Instalar las bibliotecas en DataCamp
!pip install mlxtend python-dotenv pymongo pymongo[srv]

In [1]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Reglas de asociación
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Modelos de aprendizaje automatico
from sklearn.model_selection import train_test_split

# Biblioteca para leer archivos
import os

# Biblioteca para leer .env
from dotenv import load_dotenv

# Importamos la biblioteca para MongoDB
import pymongo
from pymongo import MongoClient

# Importamos bibliotecas de lectura de APIs
import requests
import json

# Usamos el estilo de ggplot
plt.style.use("ggplot")

# Evitar que pandas trunque la visualización
# de las columnas
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# Entendimiento de los datos

Los datos fueron entregados en archivos _.csv_ separados por año. Hasta el momento se cuenta con registros de calificaciones de los años 2016 al 2021. Lo primero que debemos hacer es explorarlos, limpiarlos y arreglarlos por separado antes de consolidarlos en un mismo marco de datos para su análisis.

In [ ]:
# Lista con los nombres de archivos
archivos = ["datos_2016.csv", "datos_2017.csv", "datos_2018.csv", "datos_2019.csv", "2020_2021.csv"]
# Leemos cada archivo y lo agregamos como dataframe al diccionario
datos = {
    "2016" : pd.read_csv(archivos[0]),
    "2017" : pd.read_csv(archivos[1]),
    "2018" : pd.read_csv(archivos[2]),
    "2019" : pd.read_csv(archivos[3]),
    "2020_2021" : pd.read_csv(archivos[4])
}

Este paso es opcional, en caso que se requiera desanonimizar los datos, esta función permitirá lograrlo.

In [ ]:
# Paso opcional por si se quieren desenmascarar matriculas
def desenmascarar_matriculas(datos):
    """
    Se realiza un desenmascaramiento de matriculas
    """
    for key in datos.keys():
        n_matricula = datos[key]["n_matricula"].values - 11122
        datos[key]["n_matricula"] = n_matricula
        
    return(datos)
    
# Aplicamos el proceso de desenmascaramiento de datos
datos = desenmascarar_matriculas(datos)

## Vista previa de los datos

Para comenzar, se realiza una vista previa de los datos para cada año en el diccionario que contiene los dataframe.

Lo primero que se puede observar es que los datos corresponden a registros de calificaciones individuales de estudiantes. Estos registros se repiten múltiples veces para cada estudiante y se pueden identificar los periodos escolares a traves de una columna `periodo`.

Por otra parte, se puede observar que hay una columna con datos que parecen ser una enumeración, estos que se pueden descartar.

Tenemos datos del ceneval, separados en cuatro principales temas, ceneval_global (observese el typo), ceneval_analitico, matemático, lengua y español. También se tiene datos de los módulos que contienen información de temas adicionales.

Importante observar que los datos mantienen la misma estructura en los años 2016 al 2019.

In [ ]:
# Mostramos una vista previa de los datos
datos["2016"].head()

In [ ]:
# Mostramos una vista previa de los datos
datos["2017"].head()

In [ ]:
# Mostramos una vista previa de los datos
datos["2018"].head()

In [ ]:
# Mostramos una vista previa de los datos
datos["2019"].head()

A diferencia de sus predecesores, este conjunto de datos pertenenciente a los años 2020 y 2021 (según su nombre lo indica), tiene menos variables (columnas). Obsérvese que este conjunto contiene una nueva columna con las horas que trabajaba el estudiante cuando ingresó a la Universidad.

Las primeras dos columnas no aportan información, de hecho se tratan de simples enumeraciones que se pueden descartar. Nótese también que en este conjunto de datos muchas de las variables del ceneval han sido descartadas, así como los datos de los módulos.

La columna `carrera` fue cambiada por su equivalente a `programa` asi que habrá que considerar estandarizar un nombre para este campo ya que se consolidarán todos los datos en un mismo dataframe.

In [ ]:
# Mostramos una vista previa de los datos
datos["2020_2021"].head()

## Eliminación de variables

Antes de explorar el comportamiento de cada variable, eliminamos algunas las variables que no proporcionan información relevante como las enumeraciones de los datos y las columnas con los módulos ya que esta información se resume en los resultados del ceneval.

In [ ]:
for key in datos.keys():
    # Eliminamos la primera columna con indices
    del datos[key]["Unnamed: 0"]

    # Excluimos las variables que contienen información sobre los módulos
    datos[key] = datos[key].loc[:, ~datos[key].columns.str.contains("modulo")]

    if key == "2020_2021":
        # Si son los datos del 2020_2021 eliminamos columna `Column1`
        del datos[key]["Column1"]

Nos aseguranos que estas hayan sido eliminadas dentro de cada dataset

In [ ]:
# Revisamos que las columnas hayan sido eliminadas
for key in datos.keys():
    print(f"=== año de los datos: {key}")
    print(datos[key].columns.values, end = "\n\n")

Exploramos las variables individualmente y encontramos que hay algunas que no ofrecen información relevante para el análisis, estas variables son aquellas que apenas alcanzan un unico valor disponible, en este caso, los módulos. Por otra parte, los valores r_modulo`_n` tampoco ofrecen información relevante. Podemos descartar esas variables.

**Notese el typo en la variable `ceveval_global`**, también se corrige esta parte, así como el nombre de programa a carrera (que es como aparecen en los demás dataset antes del 2020_2021).

In [ ]:
# Corregimos los nombres de las variables ceneval_global y programa por carrera
for key in datos.keys():
    if key == "2020_2021":
        # Corregimos el typo en la variable ceneval_global
        datos[key].rename(columns = {"ceveval_global": "ceneval_global", "programa" : "carrera"}, inplace = True)
    else: 
        # Corregimos el typo en la variable ceneval_global
        datos[key].rename(columns = {"ceveval_global": "ceneval_global"}, inplace = True)
    # Verificamos los cambios
    print(f"=== Datos del año: {key} ===")
    print(datos[key].columns.values, end = "\n\n")

## Mover la columna de la matricula al principio

Como hemos visto en la estructura de los datos anteriores, la columna que representa un identificador para el registro puede ser la matricula del estudiante. Esta columna se encuentra hasta el extremo derecho así que para facilitar la lectura podemos insertarla al principio.

In [ ]:
# Reordenamos columnas para facilitar la lectura
for key in datos.keys():
    # Sacamos la columna de la matricula del dataframe actual
    primera_columna = datos[key].pop("n_matricula")
    # Reinsertamos esa columna al principio del dataframe
    datos[key].insert(0, "n_matricula", primera_columna)

Una vez realizada este arreglo, podemos verificar que los cambios se hayan efectuado correctamente.

In [ ]:
# Mostramos el orden de las columnas en cada dataframe
for key in datos.keys():
    print(f"Columnas del año: {key}")
    print(datos[key].columns.values, end = "\n\n")

## Revisión de datos repetidos 
Con la finalidad de mantener nuestros datos libres de registros redundantes (en todas las columnas), eliminarémos estos casos para su posterior análisis estadístico. 

Cabe mencionar que la eliminación de repeticiones no necesariamente tiene que ser para todas las columnas (variables), también se podrían distiguir registros repetidos por subconjuntos de variables, que veremos más adelante en la segunda revisión.

### Datos del 2016
Hacemos una exploración en los datos para encontrar registros que puedan llegar a repetirse

In [ ]:
# Encontramos las filas con registros duplicados
duplicados = datos["2016"].duplicated(keep = False)

Podemos observar que existen registros duplicados en las calificaciones de los estudiantes. Será necesario realizar un procesamiento para eliminarlos.

In [ ]:
# Mostramos las filas con registros duplicados ordenados por matricula y asignatura
# con la finalidad de identificar rápidamente los registros que se repiten
datos["2016"][duplicados].sort_values(by = ["n_matricula", "asignatura"], ascending = True)

Una opción es eliminar solo eliminar las copias extras de los registros que aparezcan con duplicados.

In [ ]:
# Eliminamos registros repetidos en el conjunto de datos (los registros que están de más)
datos["2016"].drop_duplicates(inplace = True)

Si revisamos de nuevo en busca de registros duplicados, ya no encontraremos ninguno para este año.

In [ ]:
# Obtenemos los registros duplicados
duplicados = datos["2016"].duplicated(keep = False)
# Mostramos los registros si existen
datos["2016"][duplicados].sort_values(by = ["n_matricula", "asignatura"])

## Debería imprimir nada

Realizamos el mismo trabajo en los demás datasets

### Datos del 2017

In [ ]:
# Exploramos datos del 2017 en busca de registros duplicados
duplicados = datos["2017"].duplicated(keep = False)
datos["2017"][duplicados].sort_values(by = ["n_matricula", "asignatura"], ascending = True)

In [ ]:
# Eliminamos registros repetidos en los datos del 2017
datos["2017"].drop_duplicates(inplace = True)

# Verificamos que los datos repetidos se hayan eliminado correctamente
duplicados = datos["2017"].duplicated(keep = False)
datos["2017"][duplicados].sort_values(by = ["n_matricula", "asignatura"])

### Datos del 2018

Nótese que al menos para esta primera exploración de registros duplicados, en el año 2018 no se han encontrado registros que tengan exactamente los mismos datos en todas su columnas.

In [ ]:
# Exploramos en busqueda de registros duplicados
duplicados = datos["2018"].duplicated(keep = False)
# Mostramos los registros que se duplican
datos["2018"][duplicados].sort_values(by = ["n_matricula", "asignatura"])

### Datos del 2019
 
Realizando la revisión en los datos del año 2019, tampoco se han encontrado registros duplicados.

In [ ]:
# Exploramos en busqueda de registros duplicados
duplicados = datos["2019"].duplicated(keep = False)
# Mostramos los registros que se duplican
datos["2019"][duplicados].sort_values(by = ["n_matricula", "asignatura"])

### Datos del 2020_2021
 
Realizando la revisión en los datos del año 2021, no se han encontrado registros duplicados de nueva cuenta.

In [ ]:

# Exploramos en busqueda de registros duplicados
duplicados = datos["2020_2021"].duplicated(keep = False)
# Mostramos los registros que se duplican
datos["2020_2021"][duplicados].sort_values(by = ["n_matricula", "asignatura"])

## Revisión de duplicados considerando otras variables
Para la segunda pasada, usaremos como referencia variables como: **n_matricula, periodo, clave y promedio final** ya que éstas nos permiten identificar un registro individual de calificación para un estudiante, en un periodo, y una asignatura. En caso de que se encuentren registros que se repitan en estas variables, podemos decir que se tratan del mismo registro de calificación, y por ende, hay que eliminarlos.

### Datos del 2016
Como se puede observar en la siguiente tabla, existen registros de estudiantes en donde se asignan calificaciones de asignaturas duplicadas pero que pertecen a una carrera diferente. Esto quiere decir que es hay problemas de consistencia en los datos que nos fueron proporcionados. Nótese que los registros representan a las calificaciones en las mismas asignaturas y tienen el mismo promedio, por lo que esto no se trata de un cambio de carrera por parte del estudiante.

In [ ]:
# Filtramos datos en conjunto original buscando duplicados que coincidan con tres variables: matricula, periodo y clave. Se ordenan los datos de forma descendente
duplicados = datos["2016"].duplicated(subset = ["periodo", "clave", "n_matricula", "promediofinal"], keep = False)
# Mostramos los registros duplicados de forma ordenada
datos["2016"][duplicados].sort_values(by = ["n_matricula", "periodo", "clave", "promediofinal"], ascending = False)

Entonces tenemos que tomar una decisión ya que habrá que mantener un registro u otro porque se comparten los mismos datos pero cambia la carrera. En este caso (y aunque no es exactamente el más correcto), se ha decidido mantener el primer registro que presenta duplicado y se descartan los consecuentes. Esto puede traer un problema ya que por el momento no se tienen los recursos para validar que el estudiante pertenece a una carrera determinada u otra. 

In [ ]:
# Eliminamos los registros duplicados a partir de los subconjuntos mencionados anteriormente
# manteniendo el primer registro que encuentre
datos["2016"].drop_duplicates(subset = ["periodo", "clave", "promediofinal", "n_matricula"], keep = "first", inplace = True)

# Revisamos que los registros se hayan eliminado correctamente
## Esto debería imprimir nada

# Filtramos datos en conjunto original buscando duplicados que coincidan con tres variables: matricula, periodo y clave. Se ordenan los datos de forma descendente
duplicados = datos["2016"].duplicated(subset = ["periodo", "clave", "promediofinal", "n_matricula"], keep = False)
# Mostramos los registros duplicados de forma ordenada
datos["2016"][duplicados].sort_values(by = ["n_matricula", "periodo", "clave", "promediofinal"], ascending = False)

Hacemos el mismo procedimiento con el resto de los datos para lo años posteriores.

### Datos del 2017

In [ ]:
# Filtramos datos en conjunto original buscando duplicados que coincidan con tres variables: matricula, periodo y clave. Se ordenan los datos de forma descendente
duplicados = datos["2017"].duplicated(subset = ["periodo", "clave", "n_matricula", "promediofinal"], keep = False)
# Mostramos los registros duplicados de forma ordenada
datos["2017"][duplicados].sort_values(by = ["n_matricula", "periodo", "clave", "promediofinal"], ascending = False)

Eliminamos los registros repetidos pero manteniendo la primera coincidencia

In [ ]:
# Eliminamos los registros duplicados a partir de los subconjuntos mencionados anteriormente
# manteniendo el primer registro que encuentre
datos["2017"].drop_duplicates(subset = ["periodo", "clave", "promediofinal", "n_matricula"], keep = "first", inplace = True)

# Revisamos que los registros se hayan eliminado correctamente
## Esto debería imprimir nada

# Filtramos datos en conjunto original buscando duplicados que coincidan con tres variables: matricula, periodo y clave. Se ordenan los datos de forma descendente
duplicados = datos["2017"].duplicated(subset = ["periodo", "clave", "promediofinal", "n_matricula"], keep = False)
# Mostramos los registros duplicados de forma ordenada
datos["2017"][duplicados].sort_values(by = ["n_matricula", "periodo", "clave", "promediofinal"], ascending = False)

### Datos del 2018

Buscamos los regustros duplicados

In [ ]:

# Filtramos datos en conjunto original buscando duplicados que coincidan con tres variables: matricula, periodo y clave. Se ordenan los datos de forma descendente
duplicados = datos["2018"].duplicated(subset = ["periodo", "clave", "n_matricula", "promediofinal"], keep = False)
# Mostramos los registros duplicados de forma ordenada
datos["2018"][duplicados].sort_values(by = ["n_matricula", "periodo", "clave", "promediofinal"], ascending = False)

Eliminamos los registros repetidos y mantenemos la primera coincidencia

In [ ]:
# Elimanamos los registros duplicados a partir de los subconjuntos mencionados anteriormente
# manteniendo el primer registro que encuentre
datos["2018"].drop_duplicates(subset = ["periodo", "clave", "promediofinal", "n_matricula"], keep = "first", inplace = True)

# Revisamos que los registros se hayan eliminado correctamente
## Esto debería imprimir nada

# Filtramos datos en conjunto original buscando duplicados que coincidan con tres variables: matricula, periodo y clave. Se ordenan los datos de forma descendente
duplicados = datos["2018"].duplicated(subset = ["periodo", "clave", "promediofinal", "n_matricula"], keep = False)
# Mostramos los registros duplicados de forma ordenada
datos["2018"][duplicados].sort_values(by = ["n_matricula", "periodo", "clave", "promediofinal"], ascending = False)

### Datos del 2019

In [ ]:
# Filtramos datos en conjunto original buscando duplicados que coincidan con tres variables: matricula, periodo y clave. Se ordenan los datos de forma descendente
duplicados = datos["2019"].duplicated(subset = ["periodo", "clave", "n_matricula", "promediofinal"], keep = False)
# Mostramos los registros duplicados de forma ordenada
datos["2019"][duplicados].sort_values(by = ["n_matricula", "periodo", "clave", "promediofinal"], ascending = False)

Eliminamos los registros repetidos

In [ ]:
# Eliminamos los registros duplicados a partir de los subconjuntos mencionados anteriormente
# manteniendo el primer registro que encuentre
datos["2019"].drop_duplicates(subset = ["periodo", "clave", "promediofinal", "n_matricula"], keep = "first", inplace = True)

# Revisamos que los registros se hayan eliminado correctamente
## Esto debería imprimir nada

# Filtramos datos en conjunto original buscando duplicados que coincidan con tres variables: matricula, periodo y clave. Se ordenan los datos de forma descendente
duplicados = datos["2019"].duplicated(subset = ["periodo", "clave", "promediofinal", "n_matricula"], keep = False)
# Mostramos los registros duplicados de forma ordenada
datos["2019"][duplicados].sort_values(by = ["n_matricula", "periodo", "clave", "promediofinal"], ascending = False)

### Datos del 2020_2021

En este caso particular, para la segunda pasada, no se han encontrado registros repetidos tomando como referencia el subconjunto de variables mencionadas al principio de esta sección.

In [ ]:
# Filtramos datos en conjunto original buscando duplicados que coincidan con tres variables: matricula, periodo y clave. Se ordenan los datos de forma descendente
duplicados = datos["2020_2021"].duplicated(subset = ["periodo", "clave", "n_matricula", "promediofinal"], keep = False)
# Mostramos los registros duplicados de forma ordenada
datos["2020_2021"][duplicados].sort_values(by = ["n_matricula", "periodo", "clave", "promediofinal"], ascending = False)

## Revisión de inconsistencias en los datos

Aunque ya se ha realizado una exploración de los registros duplicados, es necesario investigar si los datos cumplen con el requisito de tener máximo 9 asignaturas cargadas en un mismo periodo (para los periodos de primavera y otoño). Para lograrlo, se pueden agrupar los datos por `periodo` y `n_matricula` con la finalidad de realizar un conteo y filtrar aquellas observaciones que no cumplan con esta condición. 

### Datos del 2016

Con la finalidad de agilizar este proceso repetitivo para cada año, se ha realizado una función que permite contabilizar la cantidad de registros por periodo y matricula.

In [ ]:
def asignaturas_por_periodo(dataframe):
    """
    Permite obtener la cantidad de asignaturas por
    semestre agrupadas por periodo y matricula. 
    Devuelve el resultado como dataframe.
    """
    # Agrupamos datos por periodo y matricula para obtener la cantidad de asignaturas por grupo
    materias_por_semestre = dataframe.groupby(["periodo", "n_matricula"]).size()
    # Obtenemos las materias por semestre en un dataframe ordenadas por cantidad de asignaturas y matriculas de mayor a menor
    materias_por_semestre = materias_por_semestre.reset_index(name = "n_asignaturas").sort_values(by = ["n_asignaturas", "n_matricula"], ascending = False)
    return(materias_por_semestre)

Ahora, solo se necesita enviar el dataframe del año. Los datos se muestran ordenados de mayor a menor cantidad de asignaturas por convenciencia.

In [ ]:

# Calculamos las materias por semestre para este año
materias_por_semestre = asignaturas_por_periodo(datos["2016"])
# Mostramos el dataframe
materias_por_semestre

Como pudimos notar en la salida anterior, existen registros de estudiantes que exceden el máximo permitido de asignaturas cargadas en un semestre. Es necesario 
explorar detalladamente estos registros que son los que están causando el problema para averiguar por qué suceden estos casos.

Para agilizar el proceso de filtrado de registros por agrupamiento que no cumplen con la condición de 9 asignaturas máximas por semestre, usaremos la siguiente función que toma como parámetro el dataframe que se calculó anteriormente.

In [ ]:
def agrupaciones_inconsistentes(materias_por_semestre):
    """
    Permite filtrar las agrupaciones que tienen más de 9
    asignaturas por periodo y matricula.
    """
    # Obtenemos los registros que contiene inconsistencias
    inconsistencias = materias_por_semestre[materias_por_semestre["n_asignaturas"] > 9]
    # Cantidad de estudiantes que superan el limite permitido de asignaturas por semestre
    print("Agrupaciones con exceso de asignaturas por periodo: ", len(inconsistencias))
    # Ordenamos las inconsistencias de mayor a menor
    inconsistencias = inconsistencias.sort_values(by = ["n_matricula", "periodo", "n_asignaturas"], ascending = False)
    return(inconsistencias)

Ahora aplicamos la función y obtenemos los registros que presentan esas inconsistencias.

In [ ]:
# Calculamos las inconsistencias para el año actual
inconsistencias = agrupaciones_inconsistentes(materias_por_semestre)
# Mostramos esas inconsistencias
inconsistencias

Como pudimos ver anteriormente, tenemos 4 agrupaciones que nos arrojan 10 registros de asignaturas por semestre. En total se espera observar 40 registros (4 grupos de 10 asignaturas cada una). Filtramos los datos en el conjunto original por periodo y matrícula tomando como base los datos de las agrupaciones inconsistentes, para lograrlo ejecutamos un right join.

De igual manera, podemos hacer esta operación de filtrado mediante una función para usarla en los demás años restantes.

In [ ]:
def registros_inconsistentes(dataframe, inconsistencias):
    """
    Permite obtener los registros que corresponden a las
    agrupaciones inconsistentes mostradas anteriormente.
    """
    # Mostramos los registros de las 4 agrupaciones anteriores
    return(pd.merge(dataframe, inconsistencias[["periodo", "n_matricula"]], on = ["periodo", "n_matricula"], how = "right").sort_values(by = ["n_matricula", "clave"], ascending = False))

En el siguiente dataframe podemos observar los registros de un estudiante y encontramos que en algunos casos aparecen dos materias repetidas dos veces en el mismo periodo pero con diferentes calificaciones finales, lo que indicaría que en el semestre se reprobó la asignatura y despues se presentó un examen extraordinario (que casualmente se registra bajo el mismo periodo). Hay que tener en cuenta que el estudiante siempre reprueba una vez, pero en el segundo registro se obtiene una calificación diferente, que puede ser aprobatoria o reprobatoria.

In [ ]:
# Filtramos los registros inconsistentes para el año y los mostramos
registros_inconsistentes(datos["2016"], inconsistencias)

Ahora realizamos la misma operación para los datos de los años restantes.

### Datos del 2017

Obtenemos el dataframe con la cantidad de asignaturas por periodo y matrícula.

In [ ]:
# Calculamos las materias por semestre para este año
materias_por_semestre = asignaturas_por_periodo(datos["2017"])
# Mostramos el dataframe
materias_por_semestre

Filtramos las agrupaciones de la tabla anterior que presentan inconsistencias.

In [ ]:
# Calculamos las inconsistencias para el año actual
inconsistencias = agrupaciones_inconsistentes(materias_por_semestre)
# Mostramos esas inconsistencias
inconsistencias

Filtramos los registros en el dataset original que corresponden a las agrupaciones inconsistentes anteriores.

In [ ]:

# Filtramos los registros inconsistentes para el año y los mostramos
registros_inconsistentes(datos["2017"], inconsistencias)

### Datos del 2018

In [ ]:
# Calculamos las materias por semestre para este año
materias_por_semestre = asignaturas_por_periodo(datos["2018"])
# Mostramos el dataframe
materias_por_semestre

In [ ]:
# Calculamos las inconsistencias para el año actual
inconsistencias = agrupaciones_inconsistentes(materias_por_semestre)
# Mostramos esas inconsistencias
inconsistencias

In [ ]:
# Filtramos los registros inconsistentes para el año y los mostramos
registros_inconsistentes(datos["2018"], inconsistencias)

### Datos del 2019

In [ ]:

# Calculamos las materias por semestre para este año
materias_por_semestre = asignaturas_por_periodo(datos["2019"])
# Mostramos el dataframe
materias_por_semestre

In [ ]:
# Calculamos las inconsistencias para el año actual
inconsistencias = agrupaciones_inconsistentes(materias_por_semestre)
# Mostramos esas inconsistencias
inconsistencias

In [ ]:
# Filtramos los registros inconsistentes para el año y los mostramos
registros_inconsistentes(datos["2019"], inconsistencias)

### Datos del 2020_2021

Nótese que para los datos de este año no se cuentan con registros que excedan el límite de 9 asignaturas por periodo.

In [ ]:
# Calculamos las materias por semestre para este año
materias_por_semestre = asignaturas_por_periodo(datos["2020_2021"])
# Mostramos el dataframe
materias_por_semestre

In [ ]:
# Calculamos las inconsistencias para el año actual
inconsistencias = agrupaciones_inconsistentes(materias_por_semestre)
# Mostramos esas inconsistencias
inconsistencias

In [ ]:
# Filtramos los registros inconsistentes para el año y los mostramos
registros_inconsistentes(datos["2020_2021"], inconsistencias)

**Conclusión**: Los datos mostrados anteriormente no representan un problema, al contrario, presentan casos que sirven para identificar alumnos que han tomado exámenes extraordinarios, por lo tanto, no se necesita realizar alguna modificación. Esto sirve como un dato a tener en cuenta.

## Imputación de valores faltantes

Para esta sección es necesario explorar los conjuntos de calificaciones en busca de valores nulos, habrá que procesarlos, la estrategia dependerá en gran medida del tipo de variable que estemos trabajando, por ejemplo, una estrategia sería usar alguna medida estadística como el promedio para las variables cualitativas como el promedio y puntajes del ceneval, este promedio sería obtenido a partir de las observaciones que sí se tienen e imputarlo (asignarlo) en los registros faltantes.

Antes de realizar esta operación, vamos a consolidar los datos de todos los años en un mismo dataframe.

In [ ]:
# Hacemos una copia de los datos originales
# ya que sobreescribiré la variable datos por simplicidad
copia = datos.copy()
# Concatenamos los datos de todos los años en un mismo dataframe
datos = pd.concat([copia["2016"], copia["2017"], copia["2018"], copia["2019"], copia["2020_2021"]]).reset_index(drop = True)
# Mostramos los registros
datos

Lo primero que se hace es una función que permita obtener una tabla con un conteo de los valores faltantes en los datos

In [ ]:
def resumen_valores_faltantes(dataframe):
    """
    Esta función permite obtener una tabla con un conteo de
    los valores faltantes representados con NaN
    en cada variable (columna)
    """
    return(dataframe.isnull().sum().reset_index(name = "valores_faltantes"))

A continuación mostramos la tabla con los valores faltantes en cada variable en el dataframe.

Observese que, por ejemplo, la variable `promediofinal` contiene 77 registros faltantes. Para el caso de las demás variables como el ceneval, se observa que, al menos para el `ceneval_global` se tienen 3630 registros faltantes, en comparación con las demás variables del ceneval que tienen en totalidad 13,083 faltantes. Otra observación importante es que la variable `hrs_trabaja` es la columna que más datos faltantes tiene, esto se debe a que en los demás dataframes no se tenía y habrá que asignarles a los estudiantes que apliquen sus debidas horas de trabajo. Para el caso de los faltantes en esta variable, simplemente se asignará `No trabajaba` como el valor predeterminado.

In [ ]:

# Valores faltantes de calificaciones del 2016
resumen = resumen_valores_faltantes(datos)
resumen

El siguiente paso será determinar qué tipo de operación realizar al trabajar con estos valores faltantes de acuerdo al tipo de variable que se describe en esos datos.

Inicialmente podemos obtener los nombres de las columnas que presentan faltantes y obtenemos la información de esas variables para obtener el tipo de dato con el que se está trabajando. 

Por ejemplo, véase que las variables `promediofinal`, `ceneval_global`, `ceneval_analitico`, `ceneval_matematico`, `ceneval_lengua` y `ceneval_esp` son numéricas (cuantitativas), mientras que  `hrs_trabaja` son categorías (cualitativa). 

In [ ]:

# Obtenemos las columnas con valores faltantes
columnas_vf = resumen[resumen["valores_faltantes"] > 0]["index"].values
# Obtenemos la información de esas columnas
datos[columnas_vf].info()

In [ ]:
# Cantidad de valores unicos por variable
datos[columnas_vf].nunique().reset_index(name = "n_valores_unicos")

Explorando detalladamente los valores unicos cada una de estas variables vamos a darnos cuenta que para todas las variables, excepto `hrs_trabaja` se tiene datos numéricos y `NaNs`, sin embargo para esta última, también tenemos valores faltantes simbolizados con  el valor `-`. 

In [ ]:
for columna in columnas_vf:
    print(f"Columna: {columna}")
    print(datos[columna].unique(), end = "\n\n")

Para el caso de las calificaciones, calculamos este dato a partir del promedio tomando en consideración la clave de la asignatura, el profesor y el periodo. Esto es así por los siguientes motivos:

- Como sabemos, tenemos datos antes de la pandemia y durante la pandemia, lo cual deja en claro que las calificaciones obtenidas en estos dos casos se presentaron en condiciones diferentes, lo cual supondría una ventaja o desventaja, que habría que examinar detenidamente más adelante.

- Cada profesora o profesor tiene un diferente criterio de cómo evaluar a los estudiantes, lo cual significa que la dificultad de obtener buenas notas depende del docente y la asignatura que imparte.

In [ ]:
# Obtenemos los valores faltantes en cada cada carrera, periodo, clave y docente
agrupaciones_faltantes = datos[datos["promediofinal"].isnull()].groupby(["carrera", "periodo", "clave", "docente"]).size().reset_index(name = "calificaciones_faltantes")
agrupaciones_faltantes.sort_values(by = "docente")

In [ ]:
datos.merge(agrupaciones_faltantes.drop("calificaciones_faltantes", axis = 1), on = ["carrera", "periodo", "clave", "docente"], how = "left").sort_values("docente")

## Estructura y comportamiento de los datos

- Datos del **2016**: La siguiente información sobre los datos revela que a partir de la variable `ceneval_global` encontraremos valores nulos en los datos, exceptuando la última columna con las matrículas de los estudiantes. En total, contamos con `39,354` registros y `36` columnas con información útil (la primera corresponde al índice de la observación, asi que decidí no contarla). 

In [ ]:
datos["2016"].info()

In [ ]:
# Exploramos algunas medidas estadísticas para las 
# variables cuantitativas
datos["2016"].describe()

In [ ]:
carreras = datos["2016"]["carrera"].unique()
carreras

In [ ]:
# Obtenemos el promedio de 
promedio_carreras = datos["2016"][["carrera", "promediofinal"]].groupby("carrera").mean().reset_index()
promedio_carreras.rename(columns = {"promediofinal" : "calificacion_promedio"}, inplace = True)

In [ ]:
promedio_carreras = promedio_carreras.sort_values(by = "calificacion_promedio", ascending = False)
promedio_carreras

Se puede calcular el promedio de las calificaciones finales en cada carrera

In [ ]:
# Promedio de calificaciones por carrera de mayor a menor
sns.barplot(data = promedio_carreras, y = "carrera", x = "calificacion_promedio")

Los promedios finales se distribuyen de la siguiente manera para cada carrera. Observese los valores atípicos

In [ ]:
# Diagramas de cajas por carrera y promedio final
sns.boxplot(data = datos["2016"], x = "promediofinal", y = "carrera")

- Datos del **2017**:

In [ ]:
datos["2017"].info()

In [ ]:
# Exploramos algunas medidas estadísticas para las 
# variables cuantitativas
datos["2017"].describe()

- Datos del **2018**:

In [ ]:
datos["2018"].info()

In [ ]:
# Exploramos algunas medidas estadísticas para las 
# variables cuantitativas
datos["2018"].describe()

- Datos del **2019**:

In [ ]:
datos["2019"].info()

In [ ]:
# Exploramos algunas medidas estadísticas para las 
# variables cuantitativas
datos["2019"].describe()

- Datos del **2020 - 2021**: La siguiente información de este conjunto de datos demuestra que tenemos menos variables para analizar. A diferencia de los demas datasets, este descarta las variables relacionadas con las calificaciones del lengua, español, matemáticas y lectura. Asímismo, se integra una nueva columna nueva con información de las horas que un estudiante que un estudiante trabaja (si es el caso).

In [ ]:
datos["2020_2021"].info()

In [ ]:
# Exploramos algunas medidas estadísticas para las 
# variables cuantitativas
datos["2020_2021"].describe()

## Exploración de los datos del ceneval

Podemos analizar visualmente el comportamiento de los resultados del examen ceneval para su cuatro categorias: `ceneval_global`, `ceneval_analitico`, `ceneval_matematico`, `ceneval_lengua`, `ceneval_español`. 

In [ ]:
# Filtramos los datos que corresponden a resultados del ceneval en todas sus modalidades
# usando prefijos
datos_ceneval = datos["2016"].loc[:, datos["2016"].columns.str.startswith("ceneval")]
datos_ceneval

In [ ]:
# Imprimimos un histograma con los datos del ceneval_global
plt.hist(datos_ceneval["ceneval_global"], bins = 15)
plt.title("Puntaje en ceneval global")
plt.ylabel("Puntos")
plt.show()

In [ ]:
# Creamos un subplot
fig, axes = plt.subplots(5, figsize = (6, 25))
# Iteramos sobre cada variable del ceneval
for index, columna in enumerate(datos_ceneval.columns):
    # Asignamos un nuevo histograma al subplot
    axes[index].set_title(f"{columna}")
    axes[index].hist(datos_ceneval[columna], bins = 20)

# Mostramos el plot final
plt.show()

# Preparación de los datos

En esta seccion se crearán algunos conjuntos de variables que posteriormente usaremos como predictores para la construcción de los modelos de machine learning:

**Variables escolares**: 

- carrera (IDeIO, II, ILCS, IA, ...)
- promedio_general ()
- cantidad_materias
- cantidad_materias_superior
- periodo_ingreso
- estatus
- servicio
- practicante
- trabaja
- tasa_aprobación_carrera
- porcentaje_condicionados
- materias_tercera_vez
- materias_recursando
- permanencia_promedio_carrera
- supera_permanencia_promedio
- departamento_carrera

**Variables socioeconómicas**:

1. escuela_procedencia
2. trabaja

OBSERVACIÓN: Continuando con la exploración de los datos, los siguientes detalles que existen dentro de nuestros datos, son lo siguientes:

847 valores faltantes en 2016 de calificaciones en los cenevales

852 valores faltantes en 2017 de calificaciones en los cenevales y 1 valor faltante en promedio final

746 valores faltantes en 2018 de calificaciones en los cenevales y 16 valores faltante en promedio final

691 valores faltantes en 2019 de calificaciones en los cenevales

494 valores faltantes en 2020_2021 de calificaciones en los cenevales y 60 valores faltantes en promedio final



In [ ]:
datos["2016"].isnull().sum()

La solución a esto será lo siguientes:

Para los valores faltantes(NaN) en promedio final, la solución será sacar el promedio que todos los estudiantes sacaron en esa asignatura, una vez hecho esto se le asignará el resultado a los registros faltantes.

Para los valores faltantes(NaN) en Los examanes de ceneval, la solución será sacar el promedio de todos los cenevales y asignar este resultado a los valores vacios.

In [ ]:
#Función para asignar las calificaciones de los cenevales a los registros NAN
def promedio_cenevales(dataframe):
  #Calculando promedio de los cenevales
  prom_global = dataframe["ceneval_global"].mean()
  prom_analitico = dataframe["ceneval_analitico"].mean()
  prom_mat = dataframe["ceneval_matematico"].mean()
  prom_lengua = dataframe["ceneval_lengua"].mean()
  prom_esp = dataframe["ceneval_esp"].mean()

  #Asignando el promedio a los registros
  dataframe.loc[dataframe["ceneval_global"].isnull(),"ceneval_global"]= prom_global
  dataframe.loc[dataframe["ceneval_analitico"].isnull(),"ceneval_analitico"]= prom_analitico
  dataframe.loc[dataframe["ceneval_matematico"].isnull(),"ceneval_matematico"]= prom_mat
  dataframe.loc[dataframe["ceneval_lengua"].isnull(),"ceneval_lengua"]= prom_lengua
  dataframe.loc[dataframe["ceneval_esp"].isnull(),"ceneval_esp"]= prom_esp
  
  return(dataframe)

AÑO 2016

In [ ]:
promedio_cenevales(datos["2016"])

In [ ]:
#VERIFICANDO NAN EN LOS CENEVALES DEL AÑO 2016
#SE OBSERVA QUE YA NO HAY VALORES NAN EN ESTE DATAFRAME
datos["2016"].isnull().sum()

AÑO 2017

In [ ]:
promedio_cenevales(datos["2017"])

In [ ]:
#VERIFICANDO NAN EN LOS CENEVALES DEL AÑO 2016
#SE OBSERVA QUE YA NO HAY VALORES NAN EN ESTE DATAFRAME
datos["2017"].isnull().sum()

In [ ]:
#CONTINUANDO CON LOS DEMÁS DATAFRAME
promedio_cenevales(datos["2018"])
promedio_cenevales(datos["2019"])
#promedio_cenevales(datos["2020_2021"])

In [ ]:
datos["2018"].isnull().sum()

In [ ]:
datos["2019"].isnull().sum()

PARA LOS VALORES NAN QUE ESTÁN EN 2020_2021 SE HARÁ EL MISMO PROCESO PERO SIN USAR LA FUNCIÓN YA QUE ESTE DATAFRAME SÓLO CONTIENE CENEVAL_GLOBAL

In [ ]:
#PRIMERO VEAMOS LOS REGISTROS NAN QUE CONTIENE
datos["2020_2021"].isnull().sum()

In [ ]:
#HACEMOS EL PROCESOS DE INSERTAR PROMEDIO
prom_global = datos["2020_2021"]["ceneval_global"].mean()
datos["2020_2021"].loc[datos["2020_2021"]["ceneval_global"].isnull(),"ceneval_global"]= prom_global

In [ ]:
#REVISAMOS NUEVAMENTE, PARA CONFIRMAR QUE YA NO ESTÉN ESTOS VALORES
datos["2020_2021"].isnull().sum()

### Consolidamos la base

In [ ]:
D1 = pd.DataFrame.from_dict(datos["2020_2021"])
D2 = pd.DataFrame.from_dict(datos["datos_2016"])
D3 = pd.DataFrame.from_dict(datos["datos_2017"])
D4 = pd.DataFrame.from_dict(datos["datos_2018"])
D1 = pd.DataFrame.from_dict(datos["datos_2019"])

In [ ]:
datos = pd.concat([D1,D2,D3,D4,D5])

# Modelado de datos
## Regresión logística

In [ ]:
# Importamos las bibliotecas
# Para crear modelos de regresión logistica
from sklearn.linear_model import LogisticRegression
# Para separar los datos de entrenamiento y de prueba
from sklearn.model_selection import train_test_split

# Creamos subconjuntos de entrenamiento y prueba de forma estratificada
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y)

# Creamos una instancia para un modelo de regresion logistica
lr_model = LogisticRegression()
# Entrenamos un modelo con datos de entrenamiento
lr_model.fit(x_train, y_train)

# Calcular la precisión del modelo
accuracy = lr.score(x_test, y_test)

## KNearest Neighbors

In [ ]:
# Importamos las bibliotecas
# Para crear modelos de KNN
from sklearn.neighbors import KNeighborsClassifier
# Para separar los datos de entrenamiento y de prueba
from sklearn.model_selection import train_test_split

# Creamos subconjuntos de entrenamiento y prueba de forma estratificada
x_train, x_test, y_train, y_test = train_test_split(data, test_size = 0.3, stratify = y)

# Creamos una instancia para un modelo de vecinos más cercanos
knn = KNeighborsClassifier(n_neighbors = 3)
# Entrenamos un modelo con los datos de entrenamiento
knn.fit(x_train, y_train)

# Calcular la precisión del modelo
accuracy = knn.score(x_test, y_test)

Calcular la precisión del modelo a partir de la variación de la cantidad de vecinos

In [ ]:
# Creamos un diccionario vacio
training_accuracies = dict()
testing_accuracies = dict()
# Creamos un arreglo de 0 - 25
neighbors = np.arange(1, 26)

# Para cada cantidad de vecinos en arreglo
for n in neighbors:
    # Ajustamos el hiperparametro de numero de vecinos
    knn = KNeighborsClassifier(n_neighbors = n)
    # Entrenamos el modelo de knn
    knn.fit(x_train, y_train)
        
    # Medir precisión para cantidad de vecinos actual
    training_accuracies[n] = knn.score(x_train, y_train)
    testing_accuracies[n] = knn.score(x_test, y_test)
    
# Creamos una gráfica con los valores de precisión y número de vecinos
plt.plot(neighbors, accuracies.values(), label = "Precisión")
plt.legend()
plt.xlabel("Número de vecinos")
plt.ylabel("Precisión del modelo")
plt.show()

## Regresión lineal (en caso de usos futuros)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, test_size = 0.3, stratify = y)
linear_reg = LinearRegression()

## Exportar modelos en archivos

In [ ]:
# Modulo para guardar objetos en Python
import pickle

Para almacenar los modelos para usos futuros, usaremos la biblioteca pickle con la finalidad de serializar el objeto (el modelo), almacenarlo en un archivo, y posteriormente, deserializarlo en un entorno de producción de Python.

In [ ]:
# Guardamos el modelo de regresión logística en un archivo en disco
filename = "logistic_regression_model.sav"
pickle.dump(lr_model, open(filename, "wb"))

In [ ]:
# Cargamos el modelo a memoria desde el disco
loaded_model = pickle.load(open(filename, "rb"))
result = loaded_model.score(x_test, y_test)
print(result)

# Evaluación de modelos